# Analyze classifier performance 

In [1]:
import pandas as pd
import numpy as np
from pyprojroot import here
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report

Read annotations

In [12]:
a1 = pd.read_excel(here(r'02_Data/01_Webdata/01_Annotation/annotation_annotater1.xlsx'), sheet_name='Annotate_here')
a2 = pd.read_excel(here(r'02_Data/01_Webdata/01_Annotation/annotation_annotater2.xlsx'), sheet_name='Annotate_here')
# Reduce to first 1000 rows which were annotated only
a1 = a1
a2 = a2.iloc[:1000]
# Merge annotations
a = a2.merge(a1, on=['ID', 'Reference'], how='left')
a = a[['ID', 'Reference', 'Label_x', 'Label_y']].rename(columns={'Reference': 'text'})

In [13]:
label_map = {'p': 'problem',
             'c': 'no_problem',
             'a': 'adaption',
             'i': 'information',
              np.nan: 'unclear'}

In [14]:
a['True1'] = a.Label_x.map(label_map)
a['True2'] = a.Label_y.map(label_map)

Calculate inter-annotator agreements

In [15]:
sum(a.True1==a.True2)/len(a)

0.714

In [16]:
print(classification_report(a.True1, a.True2))

              precision    recall  f1-score   support

    adaption       0.80      0.67      0.73       215
 information       0.49      0.74      0.59       158
  no_problem       0.47      0.75      0.58        24
     problem       0.96      0.81      0.88       358
     unclear       0.60      0.59      0.59       245

    accuracy                           0.71      1000
   macro avg       0.67      0.71      0.68      1000
weighted avg       0.75      0.71      0.72      1000



In [17]:
cohen_kappa_score(a.True1, a.True2)

0.6214668029033232

Read classification results

In [18]:
pred = pd.read_csv(here(r'02_Data/01_Webdata/01_Annotation/corona_predictions_sample.csv'), sep='\t')

Merge annotation and prediction

In [19]:
cl = a.merge(pred, how='left', on=['ID'])
# Reduce to references with agreement between both annotators
cl = cl.loc[cl.True1==cl.True2]
cl.drop_duplicates(['ID', 'text_x'], inplace=True)

In [23]:
print(classification_report(cl.True1, cl.vote))

              precision    recall  f1-score   support

    adaption       0.64      0.94      0.77       144
 information       0.66      0.95      0.78       117
  no_problem       1.00      0.22      0.36        18
     problem       0.99      0.98      0.98       290
     unclear       0.93      0.30      0.45       145

    accuracy                           0.81       714
   macro avg       0.85      0.68      0.67       714
weighted avg       0.86      0.81      0.78       714

